In [23]:
# Input: standardize_dates.csv (WARN data) & PPP data from directory INPUT_DIR
# Output: merge_warn_ppp.csv to directory OUTPUT_DIR


import csv
import re
import requests
import os

import pandas as pd
from pathlib import Path

from utils import write_dict_rows_to_csv
from utils import write_rows_to_csv

USER_HOME = os.path.expanduser('~')
DEFAULT_HOME = str(Path(USER_HOME, '.warn-scraper'))
ETL_DIR = os.environ.get('WARN_ETL_DIR', DEFAULT_HOME)
WARN_DATA_PATH = str(Path(ETL_DIR, 'exports'))
WARN_CACHE_PATH = str(Path(ETL_DIR, 'cache'))
WARN_ANALYSIS_PATH = str(Path(ETL_DIR, 'analysis'))
INPUT_DIR = WARN_ANALYSIS_PATH
OUTPUT_DIR = WARN_ANALYSIS_PATH

# filenames to read from INPUT_DIR
# this is the compiled WARN file output by standardize_dates.py
WARN_FILENAME = "standardize_dates.csv"
# these are the names of the PPP files found on the sba.gov website
PPP_FILENAMES = ["public_150k_plus_211121.csv", "public_up_to_150k_1_211121.csv", "public_up_to_150k_2_211121.csv", "public_up_to_150k_3_211121.csv", "public_up_to_150k_4_211121.csv", "public_up_to_150k_5_211121.csv", "public_up_to_150k_6_211121.csv", "public_up_to_150k_7_211121.csv", "public_up_to_150k_8_211121.csv", "public_up_to_150k_9_211121.csv", "public_up_to_150k_10_211121.csv", "public_up_to_150k_11_211121.csv", "public_up_to_150k_12_211121.csv"]
OUTPUT_FILENAME = "merge_warn_ppp.csv"

Path(WARN_ANALYSIS_PATH).mkdir(parents=True, exist_ok=True)
warn_csv = f'{INPUT_DIR}/{WARN_FILENAME}'
ppp_csvs = [f'{INPUT_DIR}/{file}' for file in PPP_FILENAMES]
output_csv = f'{OUTPUT_DIR}/{OUTPUT_FILENAME}'

# names of columns
ppp_company_col = 'BorrowerName'
ppp_state_col = 'ProjectState'
warn_company_col = 'employer'
warn_state_col = 'state'

In [24]:

# TODO implement cacheing
def standardize_company(company_name, cache_dir):
    url = f'https://api.opencorporates.com/v0.4/companies/search?q={company_name}'
    cache_key = company_name
    downloaded_dir = f'{cache_dir}/{cache_key}.json'
    api_json = ""
    try:
        print(f'trying to read company {company_name} from cache...')
        # read from cache
        try:
            api_json = open_json(downloaded_dir)
        except UnicodeDecodeError:
            api_json = open_json(downloaded_dir, encoding="utf-8")

    except (FileNotFoundError, SyntaxError):
        print(f'failed to read from cache. Downloading API data for company {company_name}...')
        # cody's api key
        auth = requests.auth.HTTPBasicAuth('apikey', 'vLHe38cEAyunPAOORaxV')
        file_path = download_file(url, auth=auth, local_path=downloaded_dir)
        try:
            api_json = open_json(file_path)
        except UnicodeDecodeError:
            api_json = open_json(file_path, encoding="utf-8")
#     standardized_company_name = find_canonical(api_json)
    return api_json


def open_json(source_file, encoding=''):
    kwargs = {"newline": "", }
    # work-around for encoding differences between states
    if encoding:
        kwargs["encoding"] = encoding
    output_rows = []
    with open(source_file, **kwargs) as f:
        json_file = json.load(f)
    return json_file


def find_canonical(json):
    json = json['results']
    if json['companies']:
        for company in json['companies']:
            return evaluate_match(company)
    elif json['company']:
        return evaluate_match(company)
    else:
        print('No matching companies found.')
    return None

In [25]:
# def open_file(source_file, filename, encoding=''):
#     kwargs = {"newline": "", }
#     # work-around for encoding differences
#     if encoding:
#         kwargs["encoding"] = encoding
#     output_rows = []
#     with open(source_file, **kwargs) as f:
#         file = csv.reader(f)
#         for row in file:
#             output_rows.append(row)
#     return output_rows

def process_warn(df):
    df['company_name'] = df[warn_company_col]  # create new col
    df['company_name'] = df['company_name'].str.upper()  # to uppercase
    df['company_name'] = df['company_name'].str.replace(r'[^a-zA-Z ]', r'', regex=True)  # remove non-alphanum chars
    
#     df['canonical_company_name_warn'] = standardize_company(df[warn_company_col], WARN_CACHE_PATH)  # canonicalize
    
    df['company_name'] = df['company_name'].str[:18]  # get first 18 chars
    return df


def process_ppp(df):
    df['company_name'] = df[ppp_company_col]  # create new col
    df['company_name'] = df['company_name'].str.upper()  # to uppercase
    df['company_name'] = df['company_name'].str.replace(r'[^a-zA-Z ]', r'', regex=True)  # remove non-alphanum chars
    
#     df['canonical_company_name_ppp'] = standardize_company(df[ppp_company_col], WARN_CACHE_PATH)  # canonicalize
    
    df['company_name'] = df['company_name'].str[:18]  # get first 18 chars
    return df

def merge(df1, df2):
    # do inner join where company name, state match.
    merged_df = pd.merge(df1, df2, how='inner', left_on=['company_name', warn_state_col], right_on = ['company_name', ppp_state_col])
    return merged_df

In [26]:

# read in WARN data
try:
    warn_df = pd.read_csv(warn_csv, encoding='ISO-8859-1')
except:
    warn_df = pd.read_csv(warn_csv, encoding='utf-8')
    
# read in PPP data from multiple files
ppp_dfs = []
for file in ppp_csvs:
    try:
        file_df = pd.read_csv(file, encoding='ISO-8859-1')
    except:
        try:
            file_df = pd.read_csv(file, encoding='utf-8')
        except FileNotFoundError:
            print(f"File not found: {file}.")
            # dont try to merge this file
            continue
    ppp_dfs.append(file_df)
    file_df.head()



c:\users\cody-dellxps\.virtualenvs\warn-analysis-pyihy2i0\lib\site-packages\IPython\core\interactiveshell.py:3173: DtypeWarning: Columns (50) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
c:\users\cody-dellxps\.virtualenvs\warn-analysis-pyihy2i0\lib\site-packages\IPython\core\interactiveshell.py:3173: DtypeWarning: Columns (16,50) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [27]:
for df in ppp_dfs:
    print(df.head())

   LoanNumber DateApproved  SBAOfficeCode ProcessingMethod  \
0  9547507704   05/01/2020            464              PPP   
1  9777677704   05/01/2020            464              PPP   
2  5791407702   05/01/2020           1013              PPP   
3  6223567700   05/01/2020            920              PPP   
4  9662437702   05/01/2020            101              PPP   

               BorrowerName        BorrowerAddress      BorrowerCity  \
0     SUMTER COATINGS, INC.  2410 Highway 15 South            Sumter   
1     PLEASANT PLACES, INC.    7684 Southrail Road  North Charleston   
2   BOYER CHILDREN'S CLINIC       1850 BOYER AVE E           SEATTLE   
3  KIRTLEY CONSTRUCTION INC   1661 MARTIN RANCH RD    SAN BERNARDINO   
4              AERO BOX LLC                    NaN               NaN   

  BorrowerState BorrowerZip LoanStatusDate  ...             BusinessType  \
0           NaN  29150-9662     12/18/2020  ...              Corporation   
1           NaN  29420-9000     09/28/2021

In [28]:
# merge each ppp file into one complete ppp dataframe
# check to see that ppp_df is non-empty before trying to merge
ppp_df = pd.concat(ppp_dfs)
ppp_df.head()

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,BusinessType,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate
0,9547507704,05/01/2020,464.0,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,NaN,29150-9662,12/18/2020,...,Corporation,19248.0,Synovus Bank,COLUMBUS,GA,Unanswered,Unanswered,NaN,773553.37,11/20/2020
1,9777677704,05/01/2020,464.0,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,NaN,29420-9000,09/28/2021,...,Sole Proprietorship,19248.0,Synovus Bank,COLUMBUS,GA,Male Owned,Non-Veteran,NaN,746336.24,08/12/2021
2,5791407702,05/01/2020,1013.0,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,NaN,98112-2922,03/17/2021,...,Non-Profit Organization,9551.0,"Bank of America, National Association",CHARLOTTE,NC,Unanswered,Unanswered,Y,696677.49,02/10/2021
3,6223567700,05/01/2020,920.0,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,NaN,92407-1740,10/16/2021,...,Corporation,9551.0,"Bank of America, National Association",CHARLOTTE,NC,Unanswered,Unanswered,NaN,395264.11,09/10/2021
4,9662437702,05/01/2020,101.0,PPP,AERO BOX LLC,NaN,NaN,NaN,NaN,08/17/2021,...,NaN,57328.0,The Huntington National Bank,COLUMBUS,OH,Unanswered,Unanswered,NaN,370819.35,04/08/2021


In [29]:
# pre-process WARN, PPP data for merge
warn_df = process_warn(warn_df)
ppp_df = process_ppp(ppp_df)

# merge warn, ppp data
merged_df = merge(warn_df, ppp_df)
merged_df.head()

,state,employer,number_affected,date_received_raw,date_layoff_raw,date_closure_raw,location,parent_location,industry,notes,...,BusinessType,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate
0,AK,Doyon Drilling Inc.,304,5/1/20,4/7/20 to 5/31/20,NaN,North Slope,NaN,NaN,COVID-19; permanent,...,Corporation,56102.0,KeyBank National Association,CLEVELAND,OH,Unanswered,Unanswered,NaN,963802.85,01/20/2021
1,AK,South Restaurant & Coffeehouse,82,4/2/20,4/17/20,NaN,Anchorage,NaN,NaN,COVID-19,...,Limited Liability Company(LLC),116975.0,Northrim Bank,ANCHORAGE,AK,Female Owned,Non-Veteran,NaN,NaN,NaN
2,AK,South Restaurant & Coffeehouse,82,4/2/20,4/17/20,NaN,Anchorage,NaN,NaN,COVID-19,...,Limited Liability Company(LLC),3386.0,First National Bank Alaska,ANCHORAGE,AK,Female Owned,Non-Veteran,NaN,696703.89,01/25/2021
3,AL,National Packaging Co. Inc.,62,06/30/2021,08/07/2021,NaN,Decatur,NaN,NaN,NaN,...,Subchapter S Corporation,453618.0,Progress Bank and Trust,HUNTSVILLE,AL,Female Owned,Unanswered,NaN,1313738.65,11/05/2020
4,AL,Standard Furniture,87,10/26/2020,12/23/2020,NaN,Bay Minette,NaN,NaN,NaN,...,Corporation,434138.0,ServisFirst Bank,BIRMINGHAM,AL,Unanswered,Unanswered,NaN,612227.23,02/25/2021


In [30]:
merged_df.to_csv(output_csv)